In [ ]:
import random
import time
import math
from collections import defaultdict
from IPython.display import display, Markdown

"""
Hearts Game AI using Monte Carlo Tree Search (MCTS)

This program creates a simple AI for the card game Hearts using the MCTS algorithm.
It simulates a 4-player game, with each player represented by a simple AI agent.
Each agent makes decisions by searching through the game tree using the MCTS algorithm
and selecting the best move based on the results of the simulations.

The game state, card, and MCTS classes are implemented, along with functions to create the
initial game state, perform MCTS, and play the game.

Rules of the Game:
1. Hearts is a trick-taking game played with a standard 52-card deck.
2. The objective of the game is to have the lowest score at the end of the game.
3. Each heart card is worth 1 point, and the Queen of Spades is worth 13 points.
4. The game starts with each player receiving 13 cards.
5. The player with the 2 of Clubs starts the game by playing that card.
6. Players must follow suit (play a card of the same suit) if they can. If not, they may play any card.
7. The player with the highest card of the leading suit wins the trick and leads the next trick.
8. Players cannot lead with a heart until a heart has been "broken" (played on a previous trick).
9. At the end of each round, players receive points equal to the number of hearts and the Queen of Spades they have taken.
10. The game ends when a player reaches or exceeds 50 points. The player with the lowest score at that point wins.

Classes and Functions:
- Card: Represents individual playing cards.
- HeartsGameState: Represents the game state of Hearts, including rules, scoring, and legal moves.
- MCTSNode: Represents nodes in the MCTS search tree.
- create_initial_game_state(players): Creates the initial game state for a game of Hearts.
- mcts(root_state, iteration_limit=1000, time_limit=None): Performs MCTS, returning the best child node based on the number of visits.
- Node: Represents nodes in an alternate implementation of MCTS.
- play_hearts_mcts(state, iteration_limit=None, time_limit=None): Plays a game of Hearts using MCTS for decision making.
- get_result(self): Gets the result of a game state, added to the HeartsGameState class.

Made by Group J
"""

# Class to represent individual playing cards
class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank

    def __str__(self):
        return f"{self.rank}{self.suit}"

    def __repr__(self):
        return self.__str__()

    def __eq__(self, other):
        return self.suit == other.suit and self.rank == other.rank

    def __hash__(self):
        return hash((self.suit, self.rank))

# Class to represent and implement the game state of Hearts, including rules, scoring, and legal moves
class HeartsGameState:
    
    def __init__(
        self,
        players,
        current_player,
        hands,
        hearts_broken,
        played_cards,
        leading_suit,
        score,
    ):
        self.players = players
        self.current_player = current_player
        self.hands = hands
        self.hearts_broken = hearts_broken
        self.played_cards = played_cards
        self.leading_suit = leading_suit
        self.score = score

        
    def __str__(self):
        hands_str = "\n".join([f"{player}: {hand}" for player, hand in zip(self.players, self.hands)])
        played_cards_str = ", ".join([str(card) for card in self.played_cards])
        score_str = ", ".join([f"{player}: {score}" for player, score in self.score.items()])
        
        return (
            f"Current player: {self.players[self.current_player]}\n"
            f"Hands:\n{hands_str}\n"
            f"Hearts broken: {self.hearts_broken}\n"
            f"Played cards: {played_cards_str}\n"
            f"Leading suit: {self.leading_suit}\n"
            f"Score: {score_str}"
        )
    
    def get_legal_moves(self):
        hand = self.hands[self.current_player]
        if len(self.played_cards) == 0:
            if not self.hearts_broken:
                non_heart_cards = [card for card in hand if card.suit != "H"]
                return non_heart_cards if non_heart_cards else hand
            else:
                return hand
        else:
            cards_of_leading_suit = [
                card for card in hand if card.suit == self.leading_suit
            ]
            return cards_of_leading_suit if cards_of_leading_suit else hand

    def make_move(self, move):
        next_player = (self.current_player + 1) % len(self.players)
        next_hands = [hand.copy() for hand in self.hands]
        next_hands[self.current_player].remove(move)
        next_played_cards = self.played_cards.copy()
        next_played_cards.append(move)
        next_leading_suit = (
            self.leading_suit if self.leading_suit else move.suit
        )

        if len(next_played_cards) == len(self.players):
            trick_winner = self.get_trick_winner(next_played_cards)
            next_score = self.score.copy()
            next_score[self.players[trick_winner]] += sum(
                [card.rank for card in next_played_cards]
            )
            next_played_cards = []
            next_leading_suit = None
            next_player = trick_winner
        else:
            next_score = self.score.copy()

        next_hearts_broken = (
            self.hearts_broken
            or any(card.suit == "H" for card in next_played_cards)
        )

        return HeartsGameState(
            self.players,
            next_player,
            next_hands,
            next_hearts_broken,
            next_played_cards,
            next_leading_suit,
            next_score,
        )

    def is_terminal(self):
        return all(len(hand) == 0 for hand in self.hands)

    def get_reward(self):
        if not self.is_terminal():
            return 0
        min_score = min(self.score.values())
        return (
            1
            if self.score[self.players[self.current_player]] == min_score
            else -1
        )

    def get_last_move(self):
        return self.played_cards[-1] if self.played_cards else None

    def get_trick_winner(self, trick_cards):
        highest_rank = -1
        winner = None
        for i, card in enumerate(trick_cards):
            if (
                card.suit == self.leading_suit
                and card.rank > highest_rank
            ):
                highest_rank = card.rank
                winner = (self.current_player - i) % len(self.players)
        return winner
    
# Class to represent nodes in the MCTS search tree    
class MCTSNode:
    def __init__(self, game_state, parent=None):
        self.game_state = game_state
        self.parent = parent
        self.children = []
        self._number_of_visits = 0
        self._total_reward = 0

    def expand(self):
        # Add child nodes for unexplored actions
        legal_moves = self.game_state.get_legal_moves()
        for move in legal_moves:
            next_state = self.game_state.make_move(move)
            child_node = MCTSNode(next_state, parent=self)
            self.children.append(child_node)

    def select_child(self):
        # Select the child node with the highest UCB1 value
        return max(self.children, key=lambda node: node.ucb1())

    def update(self, reward):
        self._number_of_visits += 1
        self._total_reward += reward

    def ucb1(self): # Upper Confidence Bound 1
        # Compute the UCB1 value for this node
        if self._number_of_visits == 0:
            return float("inf")
        exploration_constant = 2  # This value can be adjusted
        exploitation = self._total_reward / self._number_of_visits
        exploration = exploration_constant * ((self.parent._number_of_visits ** 0.5) / self._number_of_visits)
        return exploitation + exploration

# Function to create the initial game state for a game of Hearts    
def create_initial_game_state(players):
    #C for Clubs, D for Diamonds, S for Spades, and H for Hearts
    deck = [Card(suit, rank) for suit in ["C", "D", "H", "S"] for rank in range(2, 15)] 
    random.shuffle(deck)
    hands = [sorted(deck[i::len(players)], key=lambda card: (card.suit, card.rank)) for i in range(len(players))]
    current_player = next(i for i, hand in enumerate(hands) if Card("C", 2) in hand)
    hearts_broken = False
    played_cards = []
    leading_suit = None
    score = defaultdict(int)
    return HeartsGameState(players, current_player, hands, hearts_broken, played_cards, leading_suit, score)

# Function to perform MCTS, returning the best child node based on the number of visits    
def mcts(root_state, iteration_limit=1000, time_limit=None):
    root = MCTSNode(root_state)
    start_time = time.time()

    while True:
        if time_limit and time.time() - start_time >= time_limit:
            break

        iteration_limit -= 1
        if iteration_limit <= 0:
            break

        # Selection
        node = root
        while len(node.children) > 0:
            node = node.select_child()

        # Expansion
        if not node.game_state.is_terminal():
            node.expand()
            node = random.choice(node.children)

        # Simulation
        current_state = node.game_state
        while not current_state.is_terminal():
            legal_moves = current_state.get_legal_moves()
            random_move = random.choice(legal_moves)
            current_state = current_state.make_move(random_move)

        # Backpropagation
        reward = current_state.get_reward()
        while node:
            node.update(reward)
            node = node.parent

    return max(root.children, key=lambda node: node._number_of_visits)


# Class to represent nodes in an alternate implementation of MCTS
class Node:
    def __init__(self, state, move=None, parent=None):
        self.state = state
        self.move = move
        self.parent = parent
        self.children = []
        self.visit_count = 0
        self.total_score = 0

    def expand(self):
        for move in self.state.get_legal_moves():
            next_state = self.state.make_move(move)
            child = Node(next_state, move=move, parent=self)
            self.children.append(child)
        return self.children

    def backpropagate(self, score):
        self.visit_count += 1
        self.total_score += score
        if self.parent:
            self.parent.backpropagate(score)

    def uct_score(self, exploration_param=1.0):
        if self.visit_count == 0:
            return float("inf")
        exploitation = self.total_score / self.visit_count
        exploration = exploration_param * math.sqrt(math.log(self.parent.visit_count) / self.visit_count)
        return exploitation + exploration

    def best_child(self, exploration_param=1.0):
        return max(self.children, key=lambda child: child.uct_score(exploration_param))

    def rollout(self):
        temp_state = self.state
        while not temp_state.is_terminal():
            legal_moves = temp_state.get_legal_moves()
            move = random.choice(legal_moves)
            temp_state = temp_state.make_move(move)
        return -temp_state.get_result()

    def traverse(self):
        current_node = self
        while current_node.children:
            current_node = current_node.best_child()
        return current_node

    
# Function to play a game of Hearts using MCTS for decision making    
def play_hearts_mcts(state, iteration_limit=None, time_limit=None):
    root = Node(state)
    root.expand()

    if iteration_limit:
        for _ in range(iteration_limit):
            leaf = root.traverse()
            simulation_result = leaf.rollout()
            leaf.backpropagate(simulation_result)
    elif time_limit:
        start_time = time.time()
        while time.time() - start_time < time_limit:
            leaf = root.traverse()
            simulation_result = leaf.rollout()
            leaf.backpropagate(simulation_result)

    move_scores = {child.move: child.visit_count for child in root.children}
    print(f"Move scores: {move_scores}")  #this line prints move scores
    
    best_move = max(root.children, key=lambda child: child.visit_count).move
    return best_move


# Function to get the result of a game state, added to the HeartsGameState class
def get_result(self):
    if not self.is_terminal():
        return 0
    min_score = min(self.score.values())
    return 1 if self.score[self.players[self.current_player]] == min_score else -1

HeartsGameState.get_result = get_result

# Player names, can be changed here if wanted. In our case i used our team members and the professor
players = ["Javier", "Professor", "Jean", "Atalia"]

# Create the initial game state
initial_state = create_initial_game_state(players)

# Start the game loop
current_state = initial_state

while not current_state.is_terminal():
    current_player_name = current_state.players[current_state.current_player]
    best_move = play_hearts_mcts(current_state, iteration_limit=1000, time_limit=None)
    
    legal_moves = current_state.get_legal_moves()
 
 #Display game over message and final scores
    display(Markdown(f"<span style='color: cyan'>{'*' * 20}</span>"))
    display(Markdown(f"<span style='color: cyan'>Current player: {current_player_name}</span>"))
    display(Markdown(f"<span style='color: green'>Best move: {best_move}</span>"))
    display(Markdown(f"<span style='color: yellow'>Legal moves: {legal_moves}</span>"))
    
    if best_move is not None:
        next_state = current_state.make_move(best_move)
        display(Markdown(f"<span style='color: magenta'>Next state:</span>"))
        display(Markdown(f"<pre><code>{next_state}</code></pre>"))
        
        current_state = next_state

display(Markdown(f"<span style='color: cyan'>{'*' * 20}</span>"))
display(Markdown(f"<span style='color: red'>Game over!</span>"))
final_scores_str = ', '.join(f'{player}: {score}' for player, score in current_state.score.items())
display(Markdown(f"<span style='color: green'>Final scores: {final_scores_str}</span>"))

# Find the winner and display in gold
min_score = min(current_state.score.values())
winners = [player for player, score in current_state.score.items() if score == min_score]
winners_str = ', '.join(winners)
display(Markdown(f"<span style='color: gold'>Winner(s): {winners_str}</span>"))




Move scores: {2C: 73, 4C: 105, 13C: 136, 3D: 10, 9D: 153, 10D: 54, 12D: 69, 14D: 21, 7S: 108, 8S: 135, 14S: 136}


<span style='color: cyan'>********************</span>

<span style='color: cyan'>Current player: Jean</span>

<span style='color: green'>Best move: 9D</span>

<span style='color: yellow'>Legal moves: [2C, 4C, 13C, 3D, 9D, 10D, 12D, 14D, 7S, 8S, 14S]</span>

...

<pre><code>Current player: Atalia
Hands:
Javier: []
Professor: []
Jean: []
Atalia: []
Hearts broken: True
Played cards: 
Leading suit: None
Score: Atalia: 133, Javier: 120, Professor: 96, Jean: 67</code></pre>

<span style='color: cyan'>********************</span>

<span style='color: red'>Game over!</span>

<span style='color: green'>Final scores: Atalia: 133, Javier: 120, Professor: 96, Jean: 67</span>

<span style='color: gold'>Winner(s): Jean</span>

1